<a href="https://colab.research.google.com/github/linus3003/PropVisionDashboard/blob/master/linkscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!pip install unidecode
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

     |████████████████████████████████| 911kB 5.6MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bion

In [2]:
!pip install -q xlrd
!git clone https://github.com/linus3003/AI-talents.git
!ls AI-talents

Cloning into 'AI-talents'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 78
Unpacking objects: 100% (81/81), done.
alldata0301.csv     alldata_final.csv	  linksandtext.csv	 links.csv
alldata.csv	    date_translation.csv  linksandtextfinal.csv  links_new.csv
alldata_final2.csv  linksandtext_2.csv	  linksandtext_new.csv	 README.md


In [3]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from itertools import chain
import time
import sys
import numpy as np
import pandas as pd
import regex as re
from IPython.display import Image
from bs4.element import Comment, Tag
import urllib.request
import unidecode

In [4]:
wd = webdriver.Chrome('chromedriver',options=chrome_options)

In [5]:
class Website:

  def __init__(self, name, url, main, list_class, unit_class ='', cookie_cl='', next_button_class=''):
    self.name = name
    self.url = url
    self.main = main
    self.list_class = list_class
    self.unit_class = unit_class
    self.cookie_cl = cookie_cl
    self.next_button_class = next_button_class
  
  def get_hl(self, driver, pages=2):
    
    house_links=[]
    land_url = ""
    driver.get(self.url)
    first = True

    for i in range(pages):

        #driver.get(self.url)
        print("scraping ",self.url," on page ",i+1 )
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        if self.name == 'wr':

          listings = soup.find_all("a", text = "Zum Projekt") #Wuestenrot, 
          page_data = [self.main+row['href'] for row in listings]

        elif self.name == 'hwg':
          
          listings = soup.select("a[href*=/wohnungsbau/neubauprojekte/]")
          page_data = [self.main+row['href'] for row in listings]
        

        else:
          
          land_url = driver.current_url
          listings_candidate = soup.find_all("a", class_=self.list_class)
          
          page_data =[]

          for l in listings_candidate:

            page = driver.get(self.main+l['href'])
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            start = soup.find("div", class_="objUnits")
            page_data.append(self.main+l['href'])

            if start is not None:
                units= start.find_all_next("a", class_ =self.unit_class)
                unitlist = [self.main+row['href'] for row in units]
                page_data.extend(unitlist)


        house_links.append(page_data)
        time.sleep(np.random.lognormal(0,0.5))
        links = list(chain.from_iterable(house_links))

        if self.next_button_class == "":
            return links

        if land_url != "":

            driver.get(land_url)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
        

        next_button = soup.find_all("a", class_=self.next_button_class)
        next_button_link = [row['href'] for row in next_button]
        

        if i< pages-1:
            if not next_button_link:
              return links
            elif len(next_button_link)==1:
                if first:
                  driver.get(next_button_link[0])
                  first = False
                else: return links

            else:
              driver.get(next_button_link[1])


    return links       
  

  def acceptCookies(self,driver):
    
    wd.get(self.url)
    try:
          driver.find_by_id(self.cookie_cl).click()
          print("button clicked")
    except Exception:
          print("No cookie window found")
          pass
    
   


In [6]:
hwg = Website('hwg','https://www.howoge.de/wohnungsbau/neubauprojekte.html', 'https://www.howoge.de','col-sm-6 col-lg-4 mb-4 item')

In [7]:
wr = Website('wr', 'https://www.whs-wuestenrot.de/Aktuelle-Neubau-Projekte.htm', 'https://www.whs-wuestenrot.de', 'object_link arrowLink' )

In [8]:
regions = ['berlin','muenchen','frankfurt','hamburg','duesseldorf','stuttgart','nuernberg','koeln','augsburg','bremen',
'dresden','duesseldorf','freiburg','graz','hannover','heilbronn','ingolstadt','innsbruck','kassel', 'koeln','leipzig',
'nuernberg','potsdam','regensburg','rhein-neckar','ruhrgebiet','salzburg','schleswig-holstein','stuttgart','wien','wuerzburg']

In [9]:
nbk_urls = ['https://www.neubaukompass.de/neubau-immobilien/'+region+'-region/' for region in regions]

In [10]:
nbks = [Website(u,u, 'https://www.neubaukompass.de', 
              'nbk-block hover:nbk-text-primary',
              'nbk-block nbk-px-1 nbk-py-2 hover:nbk-bg-gray-100',  
               'nbk-px-20 nbk-py-2 nbk-text-white nbk-bg-secondary nbk-border-solid nbk-border nbk-border-secondary nbk-rounded nbk-shadow-2xl hover:nbk-opacity-90 hover:nbk-cursor-pointer btn-cookies-accept-all',
               'nbk-flex nbk-items-center nbk-justify-center nbk-bg-red-700 nbk-border-none nbk-p-2 nbk-rounded nbk-cursor-pointer nbk-text-white nbk-font-medium nbk-appearance-none hover:nbk-bg-red-800 active:nbk-bg-red-800 focus:nbk-outline-none focus:nbk-shadow-none') for u in nbk_urls]




In [11]:
ws_dict = dict([ (w.name, w) for w in nbks ])

In [12]:
ws_dict.update({'wr': wr,'hwg':hwg})

In [ ]:
links=[]
for w in ws_dict:
  links.extend(ws_dict[w].get_hl(wd,pages=500))


scraping  https://www.neubaukompass.de/neubau-immobilien/berlin-region/  on page  1
scraping  https://www.neubaukompass.de/neubau-immobilien/berlin-region/  on page  2
scraping  https://www.neubaukompass.de/neubau-immobilien/berlin-region/  on page  3
scraping  https://www.neubaukompass.de/neubau-immobilien/berlin-region/  on page  4
scraping  https://www.neubaukompass.de/neubau-immobilien/berlin-region/  on page  5
scraping  https://www.neubaukompass.de/neubau-immobilien/berlin-region/  on page  6
scraping  https://www.neubaukompass.de/neubau-immobilien/berlin-region/  on page  7
scraping  https://www.neubaukompass.de/neubau-immobilien/berlin-region/  on page  8
scraping  https://www.neubaukompass.de/neubau-immobilien/berlin-region/  on page  9
scraping  https://www.neubaukompass.de/neubau-immobilien/berlin-region/  on page  10
scraping  https://www.neubaukompass.de/neubau-immobilien/berlin-region/  on page  11
scraping  https://www.neubaukompass.de/neubau-immobilien/berlin-region/  o

In [ ]:
links2 = list(chain.from_iterable(links))

In [ ]:
linksdf = pd.DataFrame(links,columns=['url'])

In [ ]:
linksdf.to_csv('links_new.csv')